# Simple Q&A with Qwen 7B

Notebook này hướng dẫn load và sử dụng Qwen 7B model để thực hiện Q&A đơn giản.

## Thông tin
- **Model**: Qwen/Qwen2.5-7B-Instruct
- **Task**: Question Answering


In [1]:
# Install dependencies (chạy cell này trước nếu chưa cài đặt)

import sys
import subprocess

def check_and_install(package_name):
    """Kiểm tra và cài đặt package nếu chưa có"""
    package_import = package_name.split('>=')[0].split('==')[0].split('<')[0].strip()
    
    # Đặc biệt xử lý torch - có thể có lỗi compatibility với Python 3.12
    if package_import == "torch":
        try:
            import torch
            print(f"✓ {package_name} đã được cài đặt (version: {torch.__version__})")
            return True
        except (ImportError, ValueError) as e:
            if isinstance(e, ValueError) and ("METH_CLASS" in str(e) or "METH_STATIC" in str(e)):
                print(f"⚠ {package_name} có vấn đề compatibility với Python {sys.version_info.major}.{sys.version_info.minor}")
                print("   ⚠️ LỖI: PyTorch không tương thích!")
                print("   💡 Giải pháp: Chạy cell tiếp theo (cell fix PyTorch) để cài đặt lại PyTorch")
                print("   Hoặc chạy lệnh sau trong terminal:")
                print("   !pip uninstall -y torch torchvision torchaudio")
                print("   !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
                raise ValueError("PyTorch không tương thích. Vui lòng chạy cell fix PyTorch trước.")
            else:
                print(f"⚠ {package_name} chưa được cài đặt. Đang cài đặt...")
                # Cài đặt PyTorch tương thích với Python 3.12
                if sys.version_info >= (3, 12):
                    subprocess.check_call([
                        sys.executable, "-m", "pip", "install", 
                        "--upgrade", "torch", "torchvision", "torchaudio", 
                        "--index-url", "https://download.pytorch.org/whl/cu118"
                    ])
                else:
                    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", package_name])
                print(f"✓ {package_name} đã được cài đặt")
                return False
    elif package_import == "bitsandbytes":
        # Đặc biệt xử lý bitsandbytes - cần upgrade để đảm bảo tương thích
        try:
            import bitsandbytes as bnb
            print(f"✓ {package_name} đã được cài đặt (version: {bnb.__version__})")
            # Kiểm tra xem có thể import được không
            from transformers import BitsAndBytesConfig
            print("   ✓ BitsAndBytesConfig có thể sử dụng")
            return True
        except ImportError:
            print(f"⚠ {package_name} chưa được cài đặt. Đang cài đặt...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", package_name])
            print(f"✓ {package_name} đã được cài đặt")
            return False
    else:
        # Xử lý các package khác
        try:
            __import__(package_import)
            print(f"✓ {package_name} đã được cài đặt")
            return True
        except ImportError:
            print(f"⚠ {package_name} chưa được cài đặt. Đang cài đặt...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"✓ {package_name} đã được cài đặt")
            return False

print("Đang kiểm tra và cài đặt dependencies...\n")
print(f"Python version: {sys.version}")

packages = [
    "torch",
    "transformers>=4.35.0",
    "accelerate>=0.24.0",
    "bitsandbytes>=0.41.0",  # Cần cho quantization
]

try:
    for package in packages:
        check_and_install(package)
    
    # Verify bitsandbytes after installation
    try:
        from transformers import BitsAndBytesConfig
        print("\n✓ BitsAndBytesConfig sẵn sàng cho quantization")
    except ImportError:
        print("\n⚠ BitsAndBytesConfig chưa sẵn sàng - có thể cần restart kernel")
    
    print("\n" + "="*50)
    print("✓ Tất cả dependencies đã sẵn sàng!")
    print("="*50)
except ValueError as e:
    if "PyTorch" in str(e):
        print("\n" + "="*50)
        print("❌ PyTorch không tương thích!")
        print("="*50)
        print("\nVui lòng:")
        print("1. Chạy cell tiếp theo (cell 'Fix PyTorch compatibility')")
        print("2. Sau đó restart kernel")
        print("3. Chạy lại cell này")
    else:
        raise


Đang kiểm tra và cài đặt dependencies...

Python version: 3.10.19 | packaged by conda-forge | (main, Oct 22 2025, 22:29:10) [GCC 14.3.0]
⚠ torch chưa được cài đặt. Đang cài đặt...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 71.0 MB/s  0:00:09m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 89.9 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 94.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 94.5 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 97.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 88.3 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 98.2 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 95.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 97.1 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5

✓ torch đã được cài đặt
⚠ transformers>=4.35.0 chưa được cài đặt. Đang cài đặt...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 53.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 9.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 49.6 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.7/791.7 kB 22.1 MB/s  0:00:00
  Attempting uninstall: huggingface-hub━━━━━━━━━━━━━━━━━━━━━━ 4/9 [charset_normalizer]
    Found existing installation: huggingface_hub 1.2.2━━━━━━━━ 4/9 [charset_normalizer]
    Uninstalling huggingface_hub-1.2.2:m━━━━━━━━━━━━━━━━━━━━━━ 4/9 [charset_normalizer]
      Successfully uninstalled huggingface_hub-1.2.2━━━━━━━━━━ 4/9 [charset_normalizer]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [transformers] [transformers]ub]


✓ transformers>=4.35.0 đã được cài đặt
⚠ accelerate>=0.24.0 chưa được cài đặt. Đang cài đặt...


✓ accelerate>=0.24.0 đã được cài đặt
⚠ bitsandbytes>=0.41.0 chưa được cài đặt. Đang cài đặt...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 84.2 MB/s  0:00:00 eta 0:00:01
✓ bitsandbytes>=0.41.0 đã được cài đặt



✓ BitsAndBytesConfig sẵn sàng cho quantization

✓ Tất cả dependencies đã sẵn sàng!


## Giải thích: Tại sao cần import torch và transformers?

### 1. **`torch` (PyTorch)** - Framework Deep Learning
- **Mục đích**: PyTorch là thư viện cốt lõi để làm việc với mô hình deep learning
- **Chức năng trong code này**:
  - Kiểm tra GPU/CUDA: `torch.cuda.is_available()` - xem có GPU không
  - Quản lý bộ nhớ GPU: `torch.cuda.empty_cache()` - giải phóng bộ nhớ GPU
  - Xử lý tensor: Tất cả tính toán của model đều dùng tensor của PyTorch
  - Định dạng dữ liệu: `torch.float16` - dùng để giảm bộ nhớ

### 2. **`transformers`** - Thư viện Hugging Face
- **`AutoModelForCausalLM`**: Load mô hình ngôn ngữ (như Qwen 7B)
  - Tự động nhận diện kiểu model và load đúng cách
  - Hỗ trợ quantization, device mapping
  
- **`AutoTokenizer`**: Xử lý text (tokenize/encode/decode)
  - Chuyển text thành số (tokens) để model hiểu
  - Chuyển tokens thành text khi model trả lời
  
- **`BitsAndBytesConfig`**: Cấu hình quantization (4-bit, 8-bit)
  - Giảm bộ nhớ từ ~14GB xuống ~4-5GB
  - Cho phép chạy model lớn trên GPU nhỏ hơn

### 3. **Tại sao cần kiểm tra GPU?**
- Model Qwen 7B rất lớn (~14GB), cần GPU để chạy nhanh
- Nếu không có GPU, model sẽ chạy trên CPU (rất chậm)
- Code này kiểm tra GPU để:
  - Quyết định có dùng quantization không
  - Quản lý bộ nhớ GPU hiệu quả
  - Tránh lỗi "Out of Memory"


## Bước 1: Import thư viện


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Kiểm tra GPU
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA memory free: {torch.cuda.get_device_properties(0).total_memory / 1e9 - torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
else:
    print("Sử dụng CPU")

# Hàm để clear GPU memory
def clear_gpu_memory():
    """Xóa cache GPU để giải phóng bộ nhớ"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print("✓ Đã xóa GPU cache")

# clear_gpu_memory()


CUDA available: True
CUDA device: NVIDIA GeForce RTX 4060 Ti
CUDA memory: 16.86 GB
CUDA memory free: 16.86 GB


## Bước 2: Load Model và Tokenizer


In [6]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

# Cấu hình quantization để tiết kiệm bộ nhớ
# Sử dụng 4-bit quantization để giảm memory usage từ ~14GB xuống ~4-5GB
use_quantization = True  # Đặt False nếu có đủ VRAM (>16GB)

# Kiểm tra bitsandbytes trước khi sử dụng quantization
bitsandbytes_available = False
if use_quantization and torch.cuda.is_available():
    try:
        from transformers import BitsAndBytesConfig
        import bitsandbytes as bnb
        bitsandbytes_available = True
        print("⚠️ Sử dụng 4-bit quantization để tiết kiệm bộ nhớ")
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
    except (ImportError, ModuleNotFoundError) as e:
        print("⚠️ bitsandbytes không khả dụng - tắt quantization")
        print(f"   Lỗi: {e}")
        print("   💡 Chạy cell 'Fix bitsandbytes installation' để cài đặt lại")
        print("   Hoặc đặt use_quantization = False để tiếp tục không dùng quantization")
        use_quantization = False
        quantization_config = None
        bitsandbytes_available = False
else:
    quantization_config = None
    if not torch.cuda.is_available():
        print("⚠️ Không sử dụng quantization (không có CUDA)")
    else:
        print("⚠️ Không sử dụng quantization (cần nhiều VRAM)")

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# Đảm bảo có pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("Loading model...")
try:
    # Configure device_map based on quantization
    if use_quantization and torch.cuda.is_available():
        # For quantization, use a specific GPU device to avoid CPU offloading
        # This ensures all quantized modules stay on GPU
        device_map_config = 0  # Use first GPU
    elif torch.cuda.is_available():
        # For non-quantized models, auto device_map is fine
        device_map_config = "auto"
    else:
        # No CUDA available
        device_map_config = None
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        torch_dtype=torch.float16 if (torch.cuda.is_available() and not use_quantization) else None,
        device_map=device_map_config,
        trust_remote_code=True
    )
    
    # Move to device if not using device_map
    if not torch.cuda.is_available() and device_map_config is None:
        model = model.to("cpu")
    
    print(f"✓ Model loaded: {model_name}")
    
    # Device detection
    if hasattr(model, 'hf_device_map') and model.hf_device_map:
        devices = set(model.hf_device_map.values())
        # Convert device IDs to strings (they can be int or str)
        device_strs = [str(d) for d in devices]
        print(f"✓ Model devices: {', '.join(sorted(device_strs))}")
    else:
        print(f"✓ Model device: {next(model.parameters()).device}")
    
    # Kiểm tra memory usage
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1e9
        reserved = torch.cuda.memory_reserved(0) / 1e9
        print(f"✓ GPU memory allocated: {allocated:.2f} GB")
        print(f"✓ GPU memory reserved: {reserved:.2f} GB")
        
except ImportError as e:
    if "bitsandbytes" in str(e).lower():
        print("❌ Error: bitsandbytes chưa được cài đặt hoặc có vấn đề")
        print("\n💡 Giải pháp:")
        print("1. Chạy cell 'Fix bitsandbytes installation' (cell trước đó)")
        print("2. Restart kernel")
        print("3. Chạy lại cell này")
        print("\nHoặc tắt quantization bằng cách:")
        print("   - Đặt use_quantization = False ở đầu cell này")
        print("   - Chạy lại cell")
        raise
    else:
        raise
except ValueError as e:
    error_msg = str(e)
    if "CPU or the disk" in error_msg or "dispatched on the CPU" in error_msg:
        print(f"❌ Error: {error_msg}")
        print("\n💡 Vấn đề: Model không đủ GPU RAM để load với quantization.")
        print("\n🔧 Giải pháp:")
        print("1. Tắt quantization (khuyến nghị):")
        print("   - Đặt use_quantization = False ở đầu cell này")
        print("   - Chạy lại cell (cần ~14GB VRAM)")
        print("\n2. Hoặc sử dụng model nhỏ hơn:")
        print("   - Thay đổi model_name thành 'Qwen/Qwen2.5-3B-Instruct'")
        print("   - Hoặc 'Qwen/Qwen2.5-1.5B-Instruct'")
        print("\n3. Hoặc giải phóng GPU memory:")
        print("   - Restart kernel")
        print("   - Đóng các ứng dụng khác đang dùng GPU")
    else:
        print(f"❌ Error loading model: {e}")
    raise
except Exception as e:
    print(f"❌ Error loading model: {e}")
    if use_quantization and "bitsandbytes" in str(e).lower():
        print("\n💡 Giải pháp:")
        print("1. Chạy cell 'Fix bitsandbytes installation'")
        print("2. Restart kernel")
        print("3. Chạy lại cell này")
        print("\nHoặc tắt quantization:")
        print("   - Đặt use_quantization = False ở đầu cell này")
        print("   - Chạy lại cell")
    raise

# Kiểm tra xem tokenizer và model đã được load chưa
print("\n" + "="*60)
print("KIỂM TRA LOAD MODEL")
print("="*60)
try:
    if 'tokenizer' not in globals() or tokenizer is None:
        raise NameError("tokenizer chưa được định nghĩa")
    if 'model' not in globals() or model is None:
        raise NameError("model chưa được định nghĩa")
    print("✓ Tokenizer và Model đã sẵn sàng!")
    print(f"  - Tokenizer: {type(tokenizer).__name__}")
    print(f"  - Model: {type(model).__name__}")
    print("="*60)
except NameError as e:
    print("❌ Lỗi:", str(e))
    print("\n💡 Vui lòng kiểm tra lại quá trình load model ở trên!")
    raise


⚠️ Sử dụng 4-bit quantization để tiết kiệm bộ nhớ
Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✓ Model loaded: Qwen/Qwen2.5-7B-Instruct
✓ Model devices: 0
✓ GPU memory allocated: 5.56 GB
✓ GPU memory reserved: 6.37 GB

KIỂM TRA LOAD MODEL
✓ Tokenizer và Model đã sẵn sàng!
  - Tokenizer: Qwen2TokenizerFast
  - Model: Qwen2ForCausalLM


## Bước 3: Hàm Q&A


In [7]:
def ask_question(question: str, max_new_tokens: int = 512, temperature: float = 0.7):
    """
    Hàm để hỏi đáp với model (tối ưu memory)
    
    Args:
        question: Câu hỏi cần trả lời
        max_new_tokens: Số token tối đa để generate (giảm nếu hết memory)
        temperature: Nhiệt độ cho sampling (0.0-1.0)
    """
    # Clear cache trước khi generate
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Format prompt cho Qwen Instruct model
    messages = [
        {"role": "user", "content": question}
    ]
    
    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    # Generate với các tùy chọn tối ưu memory
    try:
        with torch.no_grad():
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True if temperature > 0 else False,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                # Tối ưu memory
                use_cache=True,  # Sử dụng KV cache để tăng tốc
            )
    except torch.cuda.OutOfMemoryError as e:
        print(f"⚠️ Out of Memory! Đang thử với max_new_tokens nhỏ hơn...")
        # Clear cache và thử lại với max_new_tokens nhỏ hơn
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        # Thử lại với max_new_tokens giảm đi một nửa
        new_max_tokens = max_new_tokens // 2
        if new_max_tokens < 50:
            raise RuntimeError("Không đủ memory ngay cả với max_new_tokens nhỏ nhất. Hãy sử dụng quantization hoặc giảm kích thước model.")
        
        print(f"Thử lại với max_new_tokens={new_max_tokens}")
        with torch.no_grad():
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=new_max_tokens,
                temperature=temperature,
                do_sample=True if temperature > 0 else False,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                use_cache=True,
            )
    
    # Decode response
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # Clear cache sau khi generate
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return response

print("✓ Hàm ask_question đã sẵn sàng!")


✓ Hàm ask_question đã sẵn sàng!


## Bước 4: Test Q&A


In [8]:
# Test với một số câu hỏi đơn giản
test_questions = [
    "Xin chào! Bạn là ai?",
    "Python là gì?",
    "Giải thích ngắn gọn về machine learning",
]

print("="*60)
print("TESTING Q&A")
print("="*60)

for i, question in enumerate(test_questions, 1):
    print(f"\n[{i}] Câu hỏi: {question}")
    print("-" * 60)
    answer = ask_question(question)
    print(f"Trả lời: {answer}")
    print("="*60)


TESTING Q&A

[1] Câu hỏi: Xin chào! Bạn là ai?
------------------------------------------------------------
Trả lời: Xin chào! Tôi là Qwen, một mô hình ngôn ngữ lớn được tạo ra bởi Alibaba Cloud. Tôi có thể giúp trả lời câu hỏi, viết văn bản và trò chuyện với người dùng như bạn. Tôi rất vui được gặp bạn!

[2] Câu hỏi: Python là gì?
------------------------------------------------------------
Trả lời: Python là một ngôn ngữ lập trình phổ biến và linh hoạt được phát triển vào năm 1991 bởi Guido van Rossum. Python nổi tiếng vì cú pháp đơn giản và dễ đọc, khiến nó trở thành một lựa chọn tốt cho cả người mới bắt đầu lẫn chuyên gia lập trình.

Một số đặc điểm chính của Python bao gồm:

1. Cú pháp rõ ràng và dễ đọc.
2. Linh hoạt với nhiều kiểu dữ liệu tích hợp.
3. Có khả năng viết mã ngắn gọn và hiệu quả.
4. Có thể chạy trên nhiều nền tảng (Windows, Linux, macOS).
5. Có một thư viện lớn với nhiều công cụ và module.
6. Được sử dụng trong nhiều lĩnh vực như phát triển web, phân tích dữ liệu, tr

In [10]:
# Thử với câu hỏi của bạn
my_question = """
# HƯỚNG DẪN HỆ THỐNG
Bạn là chuyên gia tối ưu từ khóa tìm kiếm sản phẩm cho Concung(một chuỗi siêu thị mẹ và bé lớn nhất tại Việt Nam). Nhiệm vụ của bạn là biến đổi câu hỏi của khách hàng thành từ khóa tìm kiếm hiệu quả nhất và tạo message banner thân thiện.

# NHIỆM VỤ CHÍNH
1.  **Tối ưu hóa Từ khóa tìm kiếm (`keyword`):** Chuyển đổi `USER_QUERY` thành từ khóa tìm kiếm ngắn gọn, chính xác và hiệu quả nhất cho hệ thống của Concung.
2.  **Tạo Tin nhắn Banner (`message_banner`):** Soạn một tin nhắn banner thân thiện để xác nhận đã hiểu yêu cầu của khách hàng.
3.  **Tạo Tin nhắn Dự phòng (`message_no_result`):** Soạn một tin nhắn dự phòng được cá nhân hóa trong trường hợp không tìm thấy sản phẩm, giúp giữ chân khách hàng và gợi ý cải thiện tìm kiếm.

# QUY TẮC PHẠM VI XỬ LÝ (QUAN TRỌNG)
## is_in_scope = true khi QUERY thỏa mãn TẤT CẢ điều kiện sau:
1.  **Liên quan đến hệ sinh thái Con Cưng**: QUERY đề cập đến một trong các chủ đề sau:
    *   **Tìm kiếm sản phẩm**: Tìm kiếm, mua sắm, hoặc so sánh sản phẩm (ví dụ: "sữa cho bé", "tã Huggies", "kem chống hăm").
    *   **Thông tin & Hướng dẫn**: Các bài viết, cẩm nang, hoặc mẹo liên quan đến mẹ, bé, và gia đình (ví dụ: "cách chăm sóc trẻ sơ sinh", "thực đơn ăn dặm", "bí quyết giúp mẹ bầu đón Tết an toàn", "bệnh viện phụ sản uy tín").
    *   **Thông tin về Con Cưng**: Các chính sách, chương trình khuyến mãi, hoặc thông tin về công ty (ví dụ: "chính sách đổi trả", "khuyến mãi tháng 10", "giao hàng", "giao nhận").
2.  **Hợp pháp**: Nội dung không vi phạm pháp luật Việt Nam.
3.  **Không độc hại**: Không chứa nội dung phản cảm, thô tục, có hại.

## is_in_scope = false khi QUERY thuộc một trong các trường hợp sau:
- **Không liên quan đến hệ sinh thái Con Cưng**: Các câu hỏi kiến thức tổng quát không liên quan đến mẹ, bé và gia đình (ví dụ: "thủ đô của Pháp là gì", "kết quả xổ số"), tin tức không liên quan.
- **Nội dung độc hại**: Thô tục, phản cảm, kích động bạo lực, phân biệt chủng tộc.
- **Chính trị**: Quan điểm chính trị, ứng viên, chính sách, bầu cử.
- **Bất hợp pháp**: Vi phạm pháp luật, ma túy, vũ khí, hàng cấm.
- **So sánh đối thủ**: So sánh trực tiếp Con Cưng với các nền tảng khác (ví dụ: "giá ở Shopee rẻ hơn phải không").
- **Tấn công danh tiếng**: Phỉ báng cá nhân, doanh nghiệp.

# QUY TẮC TỐI ƯU TỪ KHÓA (CHỈ KHI is_in_scope = true)
## 1. Phân Tích Ý Định
- Xác định nhu cầu sản phẩm cốt lõi từ câu hỏi khách hàng
- Ưu tiên các nhóm sản phẩm chủ lực: mẹ và bé, thời trang, điện tử, gia dụng, thực phẩm

## 2. Xây Dựng Từ Khóa
- **Xử Lý Truy Vấn Về Thương Hiệu Con Cưng**: Nếu `USER_QUERY` chỉ là "Con Cưng" (hoặc các biến thể viết không dấu/sai chính tả như "concung", "con cung"), `keyword` phải là "Giới thiệu Con Cưng".
- Sử dụng tên sản phẩm cụ thể, không dùng mô tả chung chung
- Bao gồm thuộc tính liên quan: độ tuổi, thương hiệu, loại, chức năng, kích thước
- Ưu tiên tên sản phẩm hơn mô tả vấn đề
- Tối đa 20 từ
- Sử dụng thuật ngữ tiếng Việt phù hợp với thị trường Việt Nam
  - **SỬ DỤNG CONCUNG_SUGGESTIONS (CHỈ 2 TRƯỜỜNG HỢP)**:
  - **QUY TẮC NGHIÊM NGẶT (ÁP DỤNG CHO CẢ 2 TRƯỜỜNG HỢP)**:
      - **QUAN TRỌNG**: NẾU `USER_QUERY` CÓ TỪ 2 TỪ TRỞ LÊN VÀ ĐÃ RÕ NGHĨA, BỎ QUA `CONCUNG_SUGGESTIONS` (trừ trường hợp 2: sửa lỗi chính tả thương hiệu).
      - KHÔNG copy trực tiếp các cụm từ từ CONCUNG_SUGGESTIONS vào keyword
      - CHỈ dùng CONCUNG_SUGGESTIONS để hiểu ngành hàng/loại sản phẩm, sau đó tạo keyword
      - GIỮ NGUYÊN từ khóa gốc từ QUERY (bao gồm tên thương hiệu nếu có)
      - BỔ SUNG thông tin loại sản phẩm chung chung vào keyword (ví dụ: "sữa", "tã", "quần áo"), không phải các biến thể chi tiết.
      - KHÔNG sử dụng CONCUNG_SUGGESTIONS cho message_banner
  - **Trường hợp 1 — Keyword ngắn/không rõ**: Khi QUERY chỉ có 1 từ và hoàn toàn mơ hồ, không đoán được loại sản phẩm.
    - **ĐIỀU KIỆN SỬ DỤNG**: CHỈ khi QUERY = 1 từ VÀ không thể xác định được ngành hàng/loại sản phẩm
    - **MỤC ĐÍCH DUY NHẤT**: Dùng CONCUNG_SUGGESTIONS để xác định ngành hàng/loại sản phẩm trong cửa hàng Concung
    - **Ví dụ**:
      - QUERY "nan" + CONCUNG_SUGGESTIONS "sữa nan, nan 2, nan 3" → Phân tích: ngành hàng = sữa → Keyword "sữa nan cho bé"
      - QUERY "milo" + CONCUNG_SUGGESTIONS "milo 3in1, milo úc" → Phân tích: ngành hàng = sữa → Keyword "sữa milo"
      - QUERY "animo" → Phân tích: animo là thương hiệu độc quyền của Concung → Keyword "sản phẩm của Animo"
  - **Trường hợp 2 — Sửa chính tả thương hiệu (độ tin cậy cao)**: Khi QUERY có dấu hiệu sai chính tả thương hiệu VÀ tất cả mục trong CONCUNG_SUGGESTIONS cùng một thương hiệu (ví dụ đều là Ensure).
    - **MỤC ĐÍCH DUY NHẤT**: Chuẩn hóa tên thương hiệu trong keyword theo thương hiệu chung từ CONCUNG_SUGGESTIONS.
    - **Ví dụ**:
      - QUERY "en sua" + CONCUNG_SUGGESTIONS "sữa Ensure, sữa Ensure Gold, Sữa Ensure 237ml" → Keyword "sữa Ensure dinh dưỡng cho người lớn"
  - **LƯU Ý**: Nếu QUERY ≥ 2 từ hoặc đã rõ ràng về sản phẩm → BỎ QUA CONCUNG_SUGGESTIONS, ngoại trừ Trường hợp 2 (sửa chính tả thương hiệu – độ tin cậy cao)

## 3. Xử Lý Đặc Biệt Cho Sản Phẩm Mẹ và Bé
- Chuyển đổi vấn đề sức khỏe thành nhu cầu sản phẩm cụ thể
- Xác định độ tuổi/cân nặng để gợi ý size phù hợp
- Tập trung vào giải pháp sản phẩm thay vì mô tả triệu chứng

# QUY TẮC TẠO MESSAGE BANNER (CHỈ KHI is_in_scope = true)
## Cấu trúc message banner
1. **Thông điệp quan tâm**: Câu nói thấu hiểu, hỗ trợ bằng tiếng Việt (tối đa 100 ký tự)
2. **Gợi ý sản phẩm**: Kết nối tìm kiếm với nhu cầu/cảm xúc của khách hàng. Việc có nhắc tên thương hiệu hay không tuân theo các quy tắc ưu tiên dưới đây.

   - **QUY TẮC ƯU TIÊN (KIỂM TRA ĐẦU TIÊN)**:
       - **Mục đích**: Xác nhận chính xác thương hiệu khi hệ thống có độ tin cậy cao.
       - **ĐIỀU KIỆN**: Phải thỏa mãn **CẢ HAI** điều kiện sau: 1) `USER_QUERY` có chứa tên thương hiệu (hoặc biến thể sai chính tả) VÀ 2) Tất cả `CONCUNG_SUGGESTIONS` cùng trỏ về một thương hiệu duy nhất để xác nhận.

   - **QUY TẮC CHUNG (ÁP DỤNG KHI QUY TẮC ƯU TIÊN KHÔNG THỎA MÃN)**:
       - **Mục đích**: Mô tả sản phẩm chung khi không chắc chắn về thương hiệu.

   - **Sử dụng thẻ <b>**: Dùng để nhấn mạnh từ khóa có giá trị tìm kiếm cao và liên quan trực tiếp đến 'keyword' được tạo ra.
3. **Hành động bắt buộc (SÁNG TẠO BANNER MỚI)**:
   - **Nếu `QUY TẮC ƯU TIÊN` được thỏa mãn**: `message_banner` **BẮT BUỘC** phải nhắc đến tên thương hiệu đã được chuẩn hóa. Dựa vào `USER_QUERY`, hãy **sáng tạo một thông điệp tự nhiên và thân thiện có lồng ghép tên thương hiệu** thay vì dùng mẫu có sẵn. Tên thương hiệu phải được đặt trong thẻ `<b>`.
   - **Nếu `QUY TẮC ƯU TIÊN` không được thỏa mãn**: Áp dụng `QUY TẮC CHUNG`. **Phân tích sâu `USER_QUERY` để nắm bắt ý định và cảm xúc của khách hàng. Từ đó, hãy tự sáng tạo một `message_banner` hoàn toàn mới, phù hợp và độc đáo.** Banner cần tích hợp **mô tả loại sản phẩm hoặc tính năng chung** được rút ra từ `keyword` một cách tự nhiên. TUYỆT ĐỐI KHÔNG nhắc đến tên thương hiệu cụ thể và không lặp lại các mẫu câu nhàm chán.
4. **Emoji**: Một emoji phù hợp (chủ đề gia đình/sản phẩm tương ứng)
5. **Giọng điệu**: Quan tâm, hỗ trợ, đáng tin cậy, rất Việt Nam
6. **Cách xưng hô**: Sử dụng "Con Cưng" để xưng và "Ba mẹ" để gọi khách hàng

## Hướng dẫn tạo banner
- Sử dụng ngôn ngữ tiếng Việt nhẹ nhàng, quan tâm với cách xưng hô "Con Cưng" và "Ba mẹ"
- Thể hiện sự hiểu biết về nhu cầu khách hàng
- Xây dựng niềm tin và sự tự tin
- Tránh đưa ra lời khuyên y tế hoặc gây áp lực
- Kết nối với tình cảm và sự chăm sóc
- Phù hợp với văn hóa Việt Nam
- **BẮT BUỘC**: Hạn chế tối đa việc nhắc tên thương hiệu cụ thể, trừ trường hợp ngoại lệ đã nêu trên.

# QUY TẮC TẠO MESSAGE KHÔNG TÌM THẤY SẢN PHẨM (message_no_result)

## Cấu Trúc Bắt Buộc (2 Phần)
1.  **Bày Tỏ Đồng Cảm**: Luôn bắt đầu bằng một câu thể hiện sự tiếc nuối nhẹ nhàng. Hãy đa dạng cách mở đầu.
    - *Ví dụ*: "Ôi, Con Cưng chưa tìm thấy...", "Con Cưng tìm kỹ mà chưa ra...", "Hmm, Con Cưng tìm chưa ra rồi...", "Tiếc quá, Con Cưng chưa tìm được sản phẩm..."
2.  **Hướng Dẫn Giải Pháp Thông Minh**: Phân tích `USER_QUERY` để đưa ra **phương pháp** cải thiện tìm kiếm, **không gợi ý từ khóa cụ thể**.
    - **Query có thể sai chính tả**: Gợi ý "Ba mẹ kiểm tra lại tên sản phẩm nhé!".
    - **Query quá dài/chi tiết**: Gợi ý "Ba mẹ thử dùng từ khóa ngắn gọn hơn xem sao ạ!".
    - **Query quá ngắn/chung chung**: Gợi ý "Ba mẹ thử bổ sung thêm thông tin chi tiết (ví dụ: thương hiệu, độ tuổi) nhé!".
    - **Các trường hợp khác**: Gợi ý "Ba mẹ thử tìm với một từ khóa khác nhé!".

## Nguyên Tắc Vàng
- **Giọng điệu & Xưng Hô**: Luôn nhẹ nhàng, đồng cảm, hỗ trợ. Xưng hô "Con Cưng" và "Ba mẹ".
- **Độ dài**: Ngắn gọn (15-25 từ, tối đa 150 ký tự), văn phong đa dạng, không lặp lại.
- **Thái độ**: Tránh khẳng định "không có", thay bằng "chưa tìm thấy". Tuyệt đối không có hàm ý người dùng tìm sai.
- **Tên Thương Hiệu**: **TUYỆT ĐỐI KHÔNG** nhắc đến tên thương hiệu, trừ khi lặp lại chính xác `USER_QUERY` để xác nhận.
- **Nhấn mạnh**: Khi lặp lại `USER_QUERY`, đặt trong thẻ `<b>`. Ví dụ: "Con Cưng tìm mãi chưa ra <b>tã giấy Huggies</b>."
- **Kết thúc**: Luôn có một emoji thân thiện ở cuối.

# VÍ DỤ MINH HỌA

## Các trường hợp is_in_scope = true
**Đầu vào**: "bé 6 tháng bị táo bón nên uống sữa gì"
**Đầu ra**: `{"is_in_scope": true, "keyword": "sữa công thức dễ tiêu hóa cho bé 6 tháng chống táo bón", "message_banner": "Con khóc đêm nhiều phải không ba mẹ? Con Cưng tìm được những <b>sữa công thức</b> tốt nhất cho bé! 🍼", "message_no_result": "Tiếc quá, Con Cưng chưa tìm thấy sản phẩm cho bé. Ba mẹ thử tìm bằng từ khóa ngắn gọn hơn nhé! 🤔"}`

**Đầu vào**: "tã giấy huggies"
**Đầu ra**: `{"is_in_scope": true, "keyword": "tã giấy huggies cho bé", "message_banner": "Ba mẹ đang tìm <b>tã Huggies</b> phải không? Con Cưng có nhiều lựa chọn tiện lợi và an toàn cho bé yêu nhé! 👶", "message_no_result": "Con Cưng tìm mãi chưa ra <b>tã giấy Huggies</b>. Ba mẹ thử kiểm tra lại tên sản phẩm hoặc tìm với từ khóa chung hơn nhé! 🥺"}`

**Đầu vào**: "bao cao su"
**Đầu ra**: `{"is_in_scope": true, "keyword": "bao cao su an toàn", "message_banner": "Sản phẩm chính hãng, <b>chất lượng đảm bảo</b> - Con Cưng tin ba mẹ sẽ an tâm cho sức khỏe gia đình 💕", "message_no_result": "Hmm, Con Cưng chưa tìm ra <b>bao cao su</b>. Ba mẹ thử bổ sung thêm thông tin chi tiết về sản phẩm như thương hiệu mong muốn nhé! 🤗"}`

## Các trường hợp is_in_scope = false
**Đầu vào**: "thời tiết hôm nay như thế nào"
**Đầu ra**: `{"is_in_scope": false, "keyword": "", "message_banner": "", "message_no_result": "Rất tiếc, Con Cưng không có thông tin về thời tiết. Ba mẹ thử tìm kiếm một sản phẩm cụ thể nhé! 🤔", "reasoning": "Truy vấn hỏi về thời tiết, không liên quan đến sản phẩm."}`

**Đầu vào**: "Shopee có tốt hơn Concung không"
**Đầu ra**: `{"is_in_scope": false, "keyword": "", "message_banner": "", "message_no_result": "Con Cưng rất tiếc không thể so sánh với nền tảng khác. Ba mẹ cần tìm sản phẩm gì ạ? 🤗", "reasoning": "Truy vấn so sánh với đối thủ cạnh tranh."}`

**Đầu vào**: "cách nấu cháo cho bé"
**Đầu ra**: `{"is_in_scope": false, "keyword": "", "message_banner": "", "message_no_result": "Con Cưng chưa có công thức nấu ăn ạ. Ba mẹ có muốn tìm các loại cháo ăn liền dinh dưỡng cho bé không? 🍲", "reasoning": "Truy vấn hỏi về kiến thức chung, không phải tìm kiếm sản phẩm."}`


# YÊU CẦU ĐẦU RA
Chỉ trả về định dạng JSON hợp lệ:
```json
{
  "is_in_scope": boolean,
  "keyword": "từ khóa tìm kiếm được tối ưu (chỉ khi is_in_scope = true)",
  "message_banner": "banner thân thiện bằng tiếng Việt (luôn tạo khi is_in_scope = true)",
  "message_no_result": "thông báo khi không tìm thấy sản phẩm (luôn được tạo)",
  "reasoning":"lý do is_in_scope = false 10 - 20 từ"
}
```

# CÁC BƯỚC XỬ LÝ
1. **Kiểm tra phạm vi**: Đánh giá QUERY theo quy tắc `is_in_scope`.
2. **Tạo message_no_result**: Luôn tạo `message_no_result` theo các quy tắc ở trên, bất kể `is_in_scope` là gì.
3. **Nếu is_in_scope = false**: Trả về JSON với `is_in_scope=false`, `keyword=""`, `message_banner=""`, `message_no_result` đã tạo, và `reasoning` giải thích ngắn gọn.
4. **Nếu is_in_scope = true**:
   - **Phân tích QUERY**: Xác định độ chi tiết và rõ ràng của query.
   - **Xử lý CONCUNG_SUGGESTIONS** (khi cần thiết):
     - Phân tích suggestions để tìm thương hiệu chính xác và thông tin sản phẩm.
     - Ưu tiên sử dụng thương hiệu từ suggestions nếu có.
     - Giữ nguyên từ khóa cốt lõi từ QUERY.
     - Chỉ bổ sung thông tin loại sản phẩm chung từ suggestions, không thêm các chi tiết cụ thể.
     - KHÔNG sử dụng suggestions cho `message_banner`.
     - Chuẩn hóa chính tả thương hiệu (độ tin cậy cao): Nếu tất cả CONCUNG_SUGGESTIONS cùng một thương hiệu → sửa thương hiệu trong `keyword` theo tên đúng; không thêm thuộc tính mới từ CONCUNG_SUGGESTIONS.
   - Trích xuất nhu cầu sản phẩm cốt lõi.
   - Xác định thuộc tính sản phẩm (độ tuổi, size, thương hiệu...).
   - Chuyển đổi vấn đề thành giải pháp sản phẩm.
   - Tối ưu thành từ khóa tìm kiếm hiệu quả (CÓ THỂ bao gồm tên thương hiệu từ query gốc).
   - **Tạo `message_banner` theo quy trình nghiêm ngặt sau**:
     - **B1: Kiểm tra "QUY TẮC ƯU TIÊN"**. Nếu đủ điều kiện, tạo banner CÓ chứa tên thương hiệu và chuyển sang bước tiếp theo.
     - **B2: Nếu không đủ điều kiện ưu tiên**, tạo banner theo "QUY TẮC CHUNG" (tuyệt đối không chứa tên thương hiệu).
   - **CRITICAL**: Đảm bảo `message_banner` tuân thủ đúng quy trình B1-B2 ở trên. `message_no_result` chỉ được phép chứa tên thương hiệu khi trích dẫn lại `USER_QUERY` và đặt trong thẻ `<b>`.
   - Trả về JSON với `keyword`, `message_banner`, và `message_no_result` đã được tạo.
-----
                # DỮ LIỆU ĐẦU VÀO
                QUERY: đồng hồ chim nhại giọng 
                CONCUNG_SUGGESTIONS: "xe ô tô đồng hồ, vòng tay bé gái đồng hồ, vớ đồng Natri bé trai" 
"""

answer = ask_question(my_question,  temperature=0.7)
print(f"Câu hỏi: {my_question}")
print(f"\nTrả lời:\n{answer}")


⚠️ Out of Memory! Đang thử với max_new_tokens nhỏ hơn...
Thử lại với max_new_tokens=256


OutOfMemoryError: CUDA out of memory. Tried to allocate 166.00 MiB. GPU 0 has a total capacity of 15.70 GiB of which 30.81 MiB is free. Process 3141301 has 6.09 GiB memory in use. Process 3144297 has 9.58 GiB memory in use. Of the allocated memory 5.71 GiB is allocated by PyTorch, and 231.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Xóa GPU cache nếu gặp vấn đề về memory
clear_gpu_memory()

if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    print(f"GPU memory allocated: {allocated:.2f} GB")
    print(f"GPU memory reserved: {reserved:.2f} GB")
